# **Understanding the Amazon from Space**

# **Image Classification Using The Neural Network**

**Importing the libraries**

In [1]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

import gc
import numpy as np

from keras import backend as K
from sklearn.metrics import fbeta_score
from keras.layers import Conv2D, Dense, LSTM, Flatten, MaxPooling2D, BatchNormalization, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.metrics import fbeta_score
from tqdm import tqdm
from sklearn.utils import shuffle

import cv2
from PIL import Image
from tensorflow import keras
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential 

import seaborn as sns
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, History

**Loading the datasets**

In [2]:
#reading the labels

train_label = pd.read_csv('../input/planets-dataset/planet/planet/train_classes.csv')
sam_sub = pd.read_csv('../input/planets-dataset/planet/planet/sample_submission.csv')
train_label.head()

,image_name,tags
0,train_0,haze primary
1,train_1,agriculture clear primary water
2,train_2,clear primary
3,train_3,clear primary
4,train_4,agriculture clear habitation primary road


In [3]:
#onehot=OneHotEncoder()

encoder = LabelEncoder()

In [4]:
#encoder.fit(train_label['tags'])

label_maps = pd.DataFrame()
label_maps['tags'] = ['agriculture', 'artisinal_mine', 'bare_ground','blooming','blow_down','clear','cloudy','conventional_mine','cultivation','habitation','haze', 'partly_cloudy','primary','road','selective_logging','slash_burn','water']
label_maps['map'] = encoder.fit_transform(label_maps['tags'])
label_maps

,tags,map
0,agriculture,0
1,artisinal_mine,1
2,bare_ground,2
3,blooming,3
4,blow_down,4
5,clear,5
6,cloudy,6
7,conventional_mine,7
8,cultivation,8
9,habitation,9


In [5]:
#defining a dict of encoded labels

label_map = {'agriculture': 0,
 'artisinal_mine': 1,
 'bare_ground': 2,
 'blooming': 3,
 'blow_down': 4,
 'clear': 5,
 'cloudy': 6,
 'conventional_mine': 7,
 'cultivation': 8,
 'habitation': 9,
 'haze': 10,
 'partly_cloudy': 11,
 'primary': 12,
 'road': 13,
 'selective_logging': 14,
 'slash_burn': 15,
 'water': 16}


In [6]:
#loading the traing_images

X = []
Y = []
train_label = shuffle(train_label,random_state=0)

In [7]:
for image_name, tags in tqdm(train_label.values, miniters=400):
    arr = cv2.imread('../input/planets-dataset/planet/planet/train-jpg/{}.jpg'.format(image_name), cv2.IMREAD_UNCHANGED)
    targets = np.zeros(17)
    for t in tags.split(' '):
      targets[label_map[t]] = 1 
    arr = cv2.resize(arr, (64, 64))
    X.append(arr)
    Y.append(targets)

100%|██████████| 40479/40479 [04:54<00:00, 137.55it/s]


In [8]:
X = np.array(X, np.float16)/255.0

In [9]:
#splitting

X = np.array(X)
Y = np.array(Y)
x_train, x_val, y_train, y_val = train_test_split(X, Y, test_size = 0.2, shuffle = True, random_state = 1)

print(x_train.shape, y_train.shape, x_val.shape, y_val.shape)

(32383, 64, 64, 3) (32383, 17) (8096, 64, 64, 3) (8096, 17)


In [10]:
del X, Y
gc.collect()

69

In [11]:
gc.collect()

23

In [12]:
def fbeta(y_true, y_pred, threshold_shift=0):
    beta = 2

    # just in case of hipster activation at the final layer
    y_pred = K.clip(y_pred, 0, 1)

    # shifting the prediction threshold from .5 if needed
    y_pred_bin = K.round(y_pred + threshold_shift)

    tp = K.sum(K.round(y_true * y_pred_bin)) + K.epsilon()
    fp = K.sum(K.round(K.clip(y_pred_bin - y_true, 0, 1)))
    fn = K.sum(K.round(K.clip(y_true - y_pred, 0, 1)))

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)

    beta_squared = beta ** 2
    return (beta_squared + 1) * (precision * recall) / (beta_squared * precision + recall + K.epsilon())

In [13]:
from tensorflow.keras.optimizers import Adam

**Define the model**

In [14]:
model = keras.Sequential()
model.add(Conv2D(64, 5, 2, activation = "relu", input_shape = (64, 64, 3)))
model.add(MaxPooling2D())
model.add(Conv2D(128, 5, 2, activation = "relu"))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(512, activation = "relu"))
model.add(Dense(17, activation = "sigmoid"))
model.compile(loss = "binary_crossentropy", optimizer = Adam(), metrics = [fbeta])
model.fit(x_train, y_train, validation_data = (x_val, y_val), epochs = 45, batch_size = 64)

Epoch 1/45
506/506 [==============================] - 39s 75ms/step - loss: 0.2275 - fbeta: 0.6348 - val_loss: 0.2012 - val_fbeta: 0.6617
Epoch 2/45
506/506 [==============================] - 37s 72ms/step - loss: 0.1991 - fbeta: 0.6831 - val_loss: 0.1956 - val_fbeta: 0.7119
Epoch 3/45
506/506 [==============================] - 36s 72ms/step - loss: 0.1855 - fbeta: 0.7075 - val_loss: 0.1768 - val_fbeta: 0.7431
Epoch 4/45
506/506 [==============================] - 38s 74ms/step - loss: 0.1732 - fbeta: 0.7305 - val_loss: 0.1673 - val_fbeta: 0.7376
Epoch 5/45
506/506 [==============================] - 38s 74ms/step - loss: 0.1649 - fbeta: 0.7442 - val_loss: 0.1641 - val_fbeta: 0.7572
Epoch 6/45
506/506 [==============================] - 39s 77ms/step - loss: 0.1595 - fbeta: 0.7542 - val_loss: 0.1592 - val_fbeta: 0.7432
Epoch 7/45
506/506 [==============================] - 38s 75ms/step - loss: 0.1552 - fbeta: 0.7605 - val_loss: 0.1508 - val_fbeta: 0.7732
Epoch 8/45
506/506 [==============

In [15]:
gc.collect()

1767

In [16]:
#with tpu_strategy.scope():

test_loss, test_accuracy = model.evaluate(x_val, y_val)
print('Test loss: {}'.format(test_loss))
print('Test accuracy: {}'.format(test_accuracy))

253/253 [==============================] - 4s 14ms/step - loss: 0.1657 - fbeta: 0.8141
Test loss: 0.16565260291099548
Test accuracy: 0.8141382336616516


In [17]:
gc.collect()

790

In [18]:
del x_train, y_train, x_val, y_val
gc.collect()

23

In [19]:
#dividing my test_labels into two part for test-jpg and test-jpg-additional
test = sam_sub[0 : 40669]
files = sam_sub[40669 : ]

In [20]:
#with tpu_strategy.scope():  

test_img = []

for image_name, tags in tqdm(test.values, miniters=1000):
    arr = cv2.imread('../input/planets-dataset/planet/planet/test-jpg/{}.jpg'.format(image_name))
    test_img.append(cv2.resize(arr, (64, 64)))

for image_name, tags in tqdm(files.values, miniters=1000):
    arr = cv2.imread('../input/planets-dataset/test-jpg-additional/test-jpg-additional/{}.jpg'.format(image_name))
    test_img.append(cv2.resize(arr, (64, 64)))

test_img = np.array(test_img, np.float16)/255.0

100%|██████████| 20522/20522 [02:52<00:00, 118.98it/s]


In [21]:
gc.collect()

23

In [22]:
#with tpu_strategy.scope():

yres = []
predictions = model.predict(test_img, batch_size = 64, verbose = 2)
yres.append(predictions)

957/957 - 24s


In [23]:
gc.collect()

783

In [24]:
#converting my encoded labels back to it original form

sub = np.array(yres[0])
for i in range (1, len(yres)):
    sub += np.array(yres[i])
sub = pd.DataFrame(sub, columns = label_map)

In [25]:
sub

,agriculture,artisinal_mine,bare_ground,blooming,blow_down,clear,cloudy,conventional_mine,cultivation,habitation,haze,partly_cloudy,primary,road,selective_logging,slash_burn,water
0,5.592495e-03,8.263179e-10,3.859997e-04,1.491731e-02,7.123649e-04,9.998719e-01,8.183838e-06,3.104437e-10,7.943511e-04,1.980811e-03,2.283752e-03,4.468539e-06,0.999976,2.056596e-02,6.123275e-03,4.518768e-08,0.086308
1,1.869544e-02,3.432800e-10,3.891587e-04,1.461655e-02,3.719509e-03,9.996549e-01,3.845355e-07,1.436433e-09,1.268557e-02,4.359066e-03,2.818406e-04,1.686513e-04,0.999996,4.012489e-02,1.826057e-02,1.451659e-05,0.148402
2,1.674292e-07,1.580949e-18,9.034920e-12,3.515745e-12,1.578354e-15,1.428191e-08,2.355566e-11,1.444986e-10,7.062350e-09,1.735534e-11,7.445517e-08,1.000000e+00,1.000000,9.318872e-09,3.075213e-13,5.030049e-10,0.001077
3,3.339601e-02,1.902237e-11,3.895760e-04,2.139014e-02,2.321899e-03,9.787613e-01,6.599895e-07,2.912239e-11,5.998909e-03,1.231670e-03,2.950430e-04,4.204491e-02,0.999999,5.722016e-03,4.271328e-03,1.068461e-06,0.022184
4,9.382445e-07,4.978869e-13,4.399265e-06,1.431466e-12,7.743822e-20,3.388613e-03,8.101597e-01,1.529265e-12,4.420058e-07,2.222543e-05,2.462487e-07,7.334918e-03,0.217215,5.889535e-04,4.097045e-13,7.946465e-15,0.000059
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61186,4.387983e-09,8.338588e-18,1.303785e-09,1.264410e-19,3.235101e-24,1.522601e-04,9.885156e-01,2.241139e-14,1.817845e-09,2.095459e-06,1.299679e-04,5.927007e-05,0.003808,1.347370e-06,7.846432e-22,1.471455e-18,0.000001
61187,2.539521e-01,1.876977e-09,4.147029e-05,3.411279e-11,4.175857e-11,1.000000e+00,5.016917e-12,7.193935e-07,4.221722e-01,2.148151e-03,6.955881e-06,1.143151e-09,0.999985,4.791677e-03,5.080091e-09,4.099561e-06,0.999224
61188,2.992362e-03,3.170279e-12,7.703714e-05,8.234769e-03,1.929402e-03,9.991666e-01,1.718531e-07,1.544950e-12,2.021015e-03,6.729364e-04,6.316602e-04,1.113967e-04,1.000000,4.241347e-03,5.287826e-04,1.837965e-07,0.015833
61189,3.021240e-03,5.175319e-13,1.171949e-09,1.262499e-12,1.688339e-17,3.028542e-03,9.517731e-01,1.142794e-15,1.983345e-04,2.015233e-04,6.398147e-02,2.531395e-05,0.039450,3.560215e-03,1.122635e-11,3.166389e-15,0.010171


In [26]:
preds = []
for i in tqdm(range(sub.shape[0]), miniters=1000):
    a = sub.loc[[i]]
    a = a.apply(lambda x: x > 0.2, axis=1)
    a = a.transpose()
    a= a.loc[a[i] == True]
    ' '.join(list(a.index))
    preds.append(' '.join(list(a.index)))

sam_sub['tags'] = preds
sam_sub.to_csv('subm.csv', index=False)

100%|██████████| 61191/61191 [02:05<00:00, 488.95it/s]


**Finally, I have saved the final result into the sample submission file (csv format).**